# Multiple Results


In this notebook we show how you can in a single request, have the LLM model return multiple results per prompt. This is useful for running experiments where you want to evaluate the robustness of your prompt and the parameters of your config against a particular large language model.


Initial configuration for the notebook to run properly.

In [1]:
# Make sure paths are correct for the imports

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

We will load our settings and get the LLM service to use for the notebook.

In [2]:
from services import Service

from service_settings import ServiceSettings

service_settings = ServiceSettings.create()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


First, we will set up the text and chat services we will be submitting prompts to.


In [3]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureChatPromptExecutionSettings,  # noqa: F401
    AzureTextCompletion,
    OpenAIChatCompletion,
    OpenAIChatPromptExecutionSettings,  # noqa: F401
    OpenAITextCompletion,
    OpenAITextPromptExecutionSettings,  # noqa: F401
)

kernel = Kernel()

# Configure Azure LLM service
service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    oai_chat_service = OpenAIChatCompletion(
        service_id="oai_chat",
        env_file_path="../.env",
    )
    oai_text_service = OpenAITextCompletion(
        service_id="oai_text",
        env_file_path="../.env",
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    aoai_chat_service = AzureChatCompletion(
        service_id="aoai_chat",
        env_file_path="../.env",
    )
    aoai_text_service = AzureTextCompletion(
        service_id="aoai_text",
        env_file_path="../.env",
    )


Next, we'll set up the completion request settings for text completion services.


In [4]:
oai_text_prompt_execution_settings = OpenAITextPromptExecutionSettings(
    service="oai_text",
    extension_data={
        "max_tokens": 80,
        "temperature": 0.7,
        "top_p": 1,
        "frequency_penalty": 0.5,
        "presence_penalty": 0.5,
        "number_of_responses": 3,
    },
)

## Multiple Azure Open AI Text Completions

Text Compleation은 최신 모델에서 Deprecated되었으므로 SKIP

Here, we're setting up the settings for Chat completions.


In [5]:
oai_chat_prompt_execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="oai_chat",
    max_tokens=80,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=3,
)

## Multiple Azure OpenAI Chat Completions


In [6]:
az_oai_prompt_execution_settings = AzureChatPromptExecutionSettings(
    service_id="aoai_chat",
    max_tokens=80,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=3,
)

In [7]:

from semantic_kernel.contents import ChatHistory

if selectedService == Service.AzureOpenAI:
    content = (
        "내일은 멋진 날이 될 거예요, 느껴져요. 일찍 일어나서 달리기를 하고 나서..."
    )
    chat = ChatHistory()
    chat.add_user_message(content)
    results = await aoai_chat_service.get_chat_message_contents(
        chat_history=chat, settings=az_oai_prompt_execution_settings
    )

    for i, result in enumerate(results):
        print(f"Result {i + 1}: {result!s}")

Result 1: 멋진 계획이네요! 일찍 일어나서 달리기를 하면 상쾌한 기분으로 하루를 시작할 수 있을 거예요. 건강에도 좋고, 활력도 넘치게 해주죠. 달리기를 마친 후에는 가벼운 스트레칭이나 요가로 몸을 풀어주는 것도 추천드려요. 그리고 맛
Result 2: 아침을 상쾌하게 시작할 수 있겠네요. 달리기를 마친 후에는 건강한 아침 식사를 즐기고, 계획해둔 일들을 하나씩 차근차근 해보세요. 긍정적인 마음으로 하루를 시작하면 좋은 일들이 많이 생길 거예요. 내일도 화이팅입니다!
Result 3: 좋은 계획이네요! 일찍 일어나서 달리기를 하는 것은 하루를 활기차게 시작하는 데 큰 도움이 될 거예요. 달리기를 마치고 나서는 가벼운 스트레칭과 함께 건강한 아침 식사를 해보는 건 어떨까요? 그런 다음 하루 동안 하고 싶은 일들을 계획하고 차근차근 실천해


## Streaming Multiple Results

Here is an example pattern if you want to stream your multiple results. Note that this is not supported for Hugging Face text completions at this time.


In [8]:
if selectedService == Service.OpenAI:
    import os
    import time

    from IPython.display import clear_output

    # Determine the clear command based on OS
    clear_command = "cls" if os.name == "nt" else "clear"

    chat = ChatHistory()
    chat.add_user_message("what is the purpose of a rubber duck?")

    stream = oai_chat_service.get_streaming_chat_message_contents(
        chat_history=chat, settings=oai_chat_prompt_execution_settings
    )
    number_of_responses = oai_chat_prompt_execution_settings.number_of_responses
    texts = [""] * number_of_responses

    last_clear_time = time.time()
    clear_interval = 0.5  # seconds

    # Note: there are some quirks with displaying the output, which sometimes flashes and disappears.
    # This could be influenced by a few factors specific to Jupyter notebooks and asynchronous processing.
    # The following code attempts to buffer the results to avoid the output flashing on/off the screen.

    print("Waiting for results...")
    async for results in stream:
        current_time = time.time()

        # Update texts with new results
        for result in results:
            texts[result.choice_index] += str(result)

        # Clear and display output at intervals
        if current_time - last_clear_time > clear_interval:
            clear_output(wait=True)
            for idx, text in enumerate(texts):
                print(f"Result {idx + 1}: {text}")
            last_clear_time = current_time

    print("----------------------------------------")